In [90]:
class Value:
    def __init__(self, data: float, _children=(), _op=''):
        self.data = float(data)
        self.grad = float(0)
        self._prev = set(_children)
        self._backward = lambda: None

    def __repr__(self):
        return f'Value(data={self.data}, grad={self.grad})'

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)); 'Only accepting int, float'
        out = Value(self.data ** other, (self, ), '**') 
        
        def _backward():
            self.grad += other * (self.data) ** (other - 1) * out.grad

        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1
                
    

In [91]:
a = Value(2)
c = Value(3)
b = Value(4)

d = a * b
f = d + c
f

Value(data=11.0, grad=0.0)

In [92]:
f.grad = 1

In [93]:
f._backward()
d

Value(data=8.0, grad=1.0)

In [94]:
d._backward()
d

Value(data=8.0, grad=1.0)

In [95]:
c._backward()

In [96]:
d._backward()

In [97]:
a._backward()
b._backward()

In [99]:
a,b,c,d,f # checking populations of grads

(Value(data=2.0, grad=8.0),
 Value(data=4.0, grad=4.0),
 Value(data=3.0, grad=1.0),
 Value(data=8.0, grad=1.0),
 Value(data=11.0, grad=1))

In [100]:
d = Value(4)
c * d + c

Value(data=15.0, grad=0.0)

In [104]:
c = Value(3)
x = c ** 4 
x

Value(data=81.0, grad=0.0)

In [105]:
x.grad = 1
x._backward()
c._backward()
c

Value(data=3.0, grad=108.0)